In [1]:
import pandas as pd
import numpy as np


In [3]:
from pycaret.classification import *

In [4]:
from pycaret.classification import setup

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

In [45]:
#Reading the data
df = pd.read_csv(r"C:\Users\DELL\LoanPrediction\loan_train.csv")#Dataset were taken from kaggle
print(df.tail())

      Loan_ID  Gender Married Dependents Education Self_Employed  \
609  LP002978  Female      No          0  Graduate            No   
610  LP002979    Male     Yes         3+  Graduate            No   
611  LP002983    Male     Yes          1  Graduate            No   
612  LP002984    Male     Yes          2  Graduate            No   
613  LP002990  Female      No          0  Graduate           Yes   

     ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
609             2900                0.0        71.0             360.0   
610             4106                0.0        40.0             180.0   
611             8072              240.0       253.0             360.0   
612             7583                0.0       187.0             360.0   
613             4583                0.0       133.0             360.0   

     Credit_History Property_Area Loan_Status  
609             1.0         Rural           Y  
610             1.0         Rural           Y  
611     

In [47]:
#Analyzing the data
credit_history = pd.crosstab(df['Credit_History'],df['Loan_Status'], margins = True)
print(credit_history)
gender = pd.crosstab(df['Gender'],df['Loan_Status'], margins = True)
print(gender)
df['LoanAmount_log'] = np.log(df['LoanAmount'])
print(df.isnull().sum())

Loan_Status       N    Y  All
Credit_History               
0.0              82    7   89
1.0              97  378  475
All             179  385  564
Loan_Status    N    Y  All
Gender                    
Female        37   75  112
Male         150  339  489
All          187  414  601
Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
LoanAmount_log       22
dtype: int64


In [49]:
#Data Cleaning
df['Gender'].fillna(df['Gender'].mode()[0], inplace=True)
df['Self_Employed'].fillna(df['Self_Employed'].mode()[0], inplace=True)
df['Married'].fillna(df['Married'].mode()[0], inplace=True)
df['Credit_History'].fillna(df['Credit_History'].mode()[0], inplace=True)
df['Dependents'].fillna(df['Dependents'].mode()[0], inplace=True)
df['Loan_Status'].value_counts()
df['Loan_Amount_Term'].fillna(df['Loan_Amount_Term'].mode()[0], inplace=True)
df.LoanAmount = df.LoanAmount.fillna(df.LoanAmount.mean())
df.LoanAmount_log = df.LoanAmount_log.fillna(df.LoanAmount.mean())
df['TotalIncome'] = df['ApplicantIncome'] + df['CoapplicantIncome']
df['TotalIncome_log'] = np.log(df['TotalIncome'])
print(df.isnull().sum())
print(df.head())

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
LoanAmount_log       0
TotalIncome          0
TotalIncome_log      0
dtype: int64
    Loan_ID Gender Married Dependents     Education Self_Employed  \
0  LP001002   Male      No          0      Graduate            No   
1  LP001003   Male     Yes          1      Graduate            No   
2  LP001005   Male     Yes          0      Graduate           Yes   
3  LP001006   Male     Yes          0  Not Graduate            No   
4  LP001008   Male      No          0      Graduate            No   

   ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
0             5849                0.0  146.412162             360.0   
1             4583             1508.0  128.000000             

In [51]:
#Data Processing
X = df.iloc[:, np.r_[1:5, 9:11, 13:15]].values
y = df.iloc[:, 12].values

In [53]:
# print(X)
# print(y)
# print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# print(X_train.shape, X_test.shape)

In [57]:
#Label Encoding and Train-Test Split
labelencoder_X = LabelEncoder()
for i in range(0, 5):
  X_train[:, i] = labelencoder_X.fit_transform(X_train[:, i])
X_train[:, 7] = labelencoder_X.fit_transform(X_train[:, 7])

In [59]:
# print(X_train)

labelencoder_y = LabelEncoder()
y_train = labelencoder_y.fit_transform(y_train)

In [61]:
# print(y_train)

for i in range(0,5):
  X_test[:, i] = labelencoder_X.fit_transform(X_test[:, i])
X_test[:, 7] = labelencoder_X.fit_transform(X_test[:, 7])
y_test = labelencoder_y.fit_transform(y_test)

In [63]:
#Standardization
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.fit_transform(X_test)


In [65]:
#Decision Tree Classifier
DTClassifier = DecisionTreeClassifier(criterion='entropy', random_state=0)
DTClassifier.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [67]:
#Prediction and Evaluation
y_pred_dt = DTClassifier.predict(X_test)
#print(y_pred_dt)
print('The accuracy of decision tree is:', metrics.accuracy_score(y_pred_dt, y_test))

The accuracy of decision tree is: 0.6422764227642277
